# BioTranslator Capstone Project

In this project, I seeked to find if GPT Embeddings could outperform the AUROC scores presented by the BioTranslator. This Notebook contains the commands and output from the project.

All of the original code from the project can be found from the following GitHub: https://github.com/HanwenXuTHU/BioTranslatorProject

First, I ran the BioTranslator's Zero-Shot Model on the SwissProt dataset, and recreated the results from the original paper.
Next, I installed the OpenAI API into the project, and created a 
Then, I created GPT embeddings based on the GO terms present in SwissProt/go.obo
Following the creation of those embeddings, I ran the BioTranslator's Zero-Shot Model, this time using the embeddings created by GPT, and produced new AUROC scores.

This notebook contains the commands and results from the results we are comparing: the original BioTranslator embeddings vs. the GPT embeddings

*IMPORTANT* THIS CODE WILL NOT RUN WITHOUT AN OPENAI API KEY

## Creating the GPT Embeddings

After setting up an OpenAI API Key, run the following command in terminal

*Important: This will not work without the OpenAI API Key*

Next, run the gpt_embeddings file (shown below with the commands below that) to generate the GPT embeddings of the SwissProt go.obo file, and then to convert that .json file to a .pkl file for the model command

In [ ]:
import json
from openai import OpenAI
import os

client = OpenAI()

def get_embedding(text_to_embed):
    response = client.embeddings.create(
        input=text_to_embed,
        model="text-embedding-3-large"
    )
    return response.data[0].embedding

def parse_obo(filename):
    terms = []
    with open(filename, 'r') as f:
        current_term = {}
        for line in f:
            line = line.strip()
            if line == "[Term]":
                if current_term:
                    terms.append(current_term)
                current_term = {}
            elif line.startswith("id:"):
                current_term['id'] = line[len("id: "):]
            elif line.startswith("name:"):
                current_term['name'] = line[len("name: "):]
            elif line.startswith("def:"):
                def_text = line[len("def: "):]
                if def_text.startswith('"'):
                    def_text = def_text.split('"')[1]
                current_term['def'] = def_text
        if current_term:
            terms.append(current_term)
    return terms

def main():
    filename = "/homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/SwissProt/go.obo"  
    terms = parse_obo(filename)

    embeddings = {}

    for i, term in enumerate(terms):
        text_to_embed = term.get('def', '') or term.get('name', '')
        if text_to_embed:
            emb = get_embedding(text_to_embed)
            embeddings[term['id']] = emb
            print(f"Processed term {term['id']} ({i+1}/{len(terms)})")


    with open("go_embeddings.json", "w") as f:
        json.dump(embeddings, f)

    print(f"Saved embeddings for {len(embeddings)} terms to go_embeddings.json")

if __name__ == "__main__":
    main()


In [ ]:
# all necessary import statements
import json
import pickle
import torch

# Open the existing JSON file
with open("go_embeddings.json", "r") as f:
    data = json.load(f)

# Convert dict values (lists) to tensors
tensor_data = {k: torch.tensor(v) for k, v in data.items()}

# Save as PKL
with open("go_embeddings.pkl", "wb") as f:
    pickle.dump(tensor_data, f)


In [ ]:
%cd BioTranslatorProject/Protein
!export OPENAI_API_KEY="your key here"
!python gpt_embeddings.py
!python gpt_generator.py

## Commands

Make sure to run these commands in the BioTranslatorProject/Protein folder, like the ones above.
 The terminal output and organized results of these commands are displayed below in the results section

## BioTranslator Command for Zero-Shot Protein Prediction on SwissProt

In [ ]:
!python BioTranslatorProject/Protein/main.py --method BioTranslator --dataset SwissProt --data_repo /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/ --task zero_shot --encoder_path /homes/iws/chaafen/428/BioTranslatorProject/TextEncoder/encoder.pth --emb_path /homes/iws/chaafen/428/BioTranslatorProject/embeddings

### GPT Embedding Command for Zero-Shot Protein Prediction on SwissProt

In [ ]:
!python BioTranslatorProject/Protein/main.py --method BioTranslator --dataset SwissProt --data_repo /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/ --task zero_shot --encoder_path /homes/iws/chaafen/428/BioTranslatorProject/TextEncoder/encoder.pth --emb_path /homes/iws/chaafen/428/BioTranslatorProject/embeddings --working_space /homes/iws/chaafen/428/BioTranslatorProject/working_space --save_path /homes/iws/chaafen/428/BioTranslatorProject/results --emb_name go_embeddings.pkl

# Results

### BioTranslator AUROC Scores

| Ontology | Fold 0 | Fold 1 | Fold 2 | AVG AUROC |
|----------|--------|--------|--------|-----------|
| BP       | 0.7631 | 0.7607 | 0.7817 | 0.7686    |
| MF       | 0.7959 | 0.8024 | 0.8129 | 0.8038    |
| CC       | 0.8185 | 0.8224 | 0.8287 | 0.8232    |

### GPT Embedding AUROC Scores

| Ontology | Fold 0 | Fold 1 | Fold 2 | AVG AUROC |
|----------|--------|--------|--------|-----------|
| BP       | 0.7683 | 0.7670 | 0.7852 | 0.7735    |
| MF       | 0.8066 | 0.8030 | 0.8163 | 0.8086    |
| CC       | 0.8186 | 0.8281 | 0.8312 | 0.8260    |

### Difference

| Ontology | Original | GPT-Based | Δ       |
|----------|----------|-----------|---------|
| BP       | 0.7686   | 0.7735    | +0.0049 |
| MF       | 0.8038   | 0.8086    | +0.0048 |
| CC       | 0.8232   | 0.8260    | +0.0034 |


## Terminal Outputs

### BioTranslator

[chaafen@hamachi BioTranslatorProject]$ python Protein/main.py --method BioTranslator --dataset SwissProt --data_repo /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/ --task zero_shot --encoder_path /homes/iws/chaafen/428/BioTranslatorProject/TextEncoder/encoder.pth  --emb_path /homes/iws/chaafen/428/BioTranslatorProject/embeddings
2.7.0+cu126
True
Using GPU: 0
Loading terms file from: /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/SwissProt/terms.pkl
terms number:19361
100%|██████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 697.73it/s]
Fold 0 contains 3010 zero shot terms
Fold 1 contains 2925 zero shot terms
Fold 2 contains 2977 zero shot terms
100%|███████████████████████████████████████████████████████████████████| 47340/47340 [00:00<00:00, 226134.04it/s]
100%|██████████████████████████████████████████████████████████████████████| 47340/47340 [05:28<00:00, 143.90it/s]
Rank of your embeddings is 768
Data Loading Finished!
Start Cross-Validation Fold :0 ...
initialize network with xavier
Training: 100%|█████████████████████████████| 30/30 [30:02<00:00, 60.07s/it, Fold=0, epoch=29, train loss=0.00129]
Evaluate Our Model ...
2025-05-29 15:52:48,221 - BioTrainer.py[line:119] - INFO: fold:0 ont: bp roc auc:0.7630615442484475
2025-05-29 15:52:48,221 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 74.55890722823027% terms is greater than 0.65
2025-05-29 15:52:48,221 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 69.32270916334737% terms is greater than 0.7
2025-05-29 15:52:48,221 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 62.49288560045644% terms is greater than 0.75
2025-05-29 15:52:48,222 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 55.49231644849256% terms is greater than 0.8
2025-05-29 15:52:48,222 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 46.38588503130377% terms is greater than 0.85
2025-05-29 15:52:48,222 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 23.676721684689753% terms is greater than 0.95
2025-05-29 15:52:50,961 - BioTrainer.py[line:119] - INFO: fold:0 ont: mf roc auc:0.7959243482008153
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 77.56892230576447% terms is greater than 0.65
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 72.9323308270679% terms is greater than 0.7
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 67.54385964912323% terms is greater than 0.75
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 62.907268170426605% terms is greater than 0.8
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 55.76441102756932% terms is greater than 0.85
2025-05-29 15:52:50,961 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 31.829573934837008% terms is greater than 0.95
2025-05-29 15:52:52,511 - BioTrainer.py[line:119] - INFO: fold:0 ont: cc roc auc:0.818593567051305
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 83.51648351648326% terms is greater than 0.65
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 79.3406593406591% terms is greater than 0.7
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 74.50549450549428% terms is greater than 0.75
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 67.6923076923075% terms is greater than 0.8
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 58.46153846153829% terms is greater than 0.85
2025-05-29 15:52:52,511 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 32.747252747252666% terms is greater than 0.95
Start Cross-Validation Fold :1 ...
initialize network with xavier
Training: 100%|██████████████████████████████████████████████| 30/30 [29:25<00:00, 58.86s/it, Fold=1, epoch=29, train loss=0.0013]
Evaluate Our Model ...
2025-05-29 16:22:53,382 - BioTrainer.py[line:119] - INFO: fold:1 ont: bp roc auc:0.7607315237140234
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 71.96374622356653% terms is greater than 0.65
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 67.19033232628543% terms is greater than 0.7
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 61.812688821753554% terms is greater than 0.75
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 53.897280966768434% terms is greater than 0.8
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 45.80060422960808% terms is greater than 0.85
2025-05-29 16:22:53,383 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 23.867069486405043% terms is greater than 0.95
2025-05-29 16:22:56,260 - BioTrainer.py[line:119] - INFO: fold:1 ont: mf roc auc:0.8024333319756641
2025-05-29 16:22:56,260 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 79.85697258641119% terms is greater than 0.65
2025-05-29 16:22:56,260 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 73.89749702026117% terms is greater than 0.7
2025-05-29 16:22:56,260 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 67.46126340881915% terms is greater than 0.75
2025-05-29 16:22:56,260 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 61.62097735399213% terms is greater than 0.8
2025-05-29 16:22:56,260 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 55.06555423122711% terms is greater than 0.85
2025-05-29 16:22:56,261 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 33.969010727056045% terms is greater than 0.95
2025-05-29 16:22:57,745 - BioTrainer.py[line:119] - INFO: fold:1 ont: cc roc auc:0.822417496877089
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 81.67053364269175% terms is greater than 0.65
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 77.95823665893302% terms is greater than 0.7
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 73.54988399071952% terms is greater than 0.75
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 66.12529002320206% terms is greater than 0.8
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 57.77262180974491% terms is greater than 0.85
2025-05-29 16:22:57,745 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 33.642691415313145% terms is greater than 0.95
Start Cross-Validation Fold :2 ...
initialize network with xavier
Training: 100%|███████████████████████████████████████████████████████| 30/30 [29:38<00:00, 59.27s/it, Fold=2, epoch=29, train loss=0.00129]
Evaluate Our Model ...
2025-05-29 16:53:10,092 - BioTrainer.py[line:119] - INFO: fold:2 ont: bp roc auc:0.7817562296077228
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 78.55917667238488% terms is greater than 0.65
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 72.78444825614697% terms is greater than 0.7
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 66.32361349342536% terms is greater than 0.75
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 58.37621497998903% terms is greater than 0.8
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 49.22813036020621% terms is greater than 0.85
2025-05-29 16:53:10,092 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 25.95769010863366% terms is greater than 0.95
2025-05-29 16:53:12,887 - BioTrainer.py[line:119] - INFO: fold:2 ont: mf roc auc:0.8129072282605443
2025-05-29 16:53:12,887 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 81.80708180708153% terms is greater than 0.65
2025-05-29 16:53:12,887 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 76.92307692307666% terms is greater than 0.7
2025-05-29 16:53:12,887 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 71.79487179487155% terms is greater than 0.75
2025-05-29 16:53:12,887 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 63.98046398046377% terms is greater than 0.8
2025-05-29 16:53:12,888 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 56.410256410256224% terms is greater than 0.85
2025-05-29 16:53:12,888 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 33.94383394383384% terms is greater than 0.95
2025-05-29 16:53:14,290 - BioTrainer.py[line:119] - INFO: fold:2 ont: cc roc auc:0.8287138381916953
2025-05-29 16:53:14,290 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 83.61858190708985% terms is greater than 0.65
2025-05-29 16:53:14,290 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 78.97310513447377% terms is greater than 0.7
2025-05-29 16:53:14,290 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 74.81662591686991% terms is greater than 0.75
2025-05-29 16:53:14,290 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 69.92665036674772% terms is greater than 0.8
2025-05-29 16:53:14,290 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 60.14669926650335% terms is greater than 0.85
2025-05-29 16:53:14,291 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 36.67481662591685% terms is greater than 0.95

### GPT-Embedding

[chaafen@hamachi Protein]$ python main.py   --method BioTranslator   --dataset SwissProt   --data_repo /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/   --task zero_shot   --encoder_path /homes/iws/chaafen/428/BioTranslatorProject/TextEncoder/encoder.pth   --emb_path /homes/iws/chaafen/428/BioTranslatorProject/embeddings   --working_space /homes/iws/chaafen/428/BioTranslatorProject/working_space   --save_path /homes/iws/chaafen/428/BioTranslatorProject/results   --emb_name go_embeddings.pkl
2.7.0+cu126
True
Using GPU: 0
Loading terms file from: /homes/iws/chaafen/428/BioTranslatorProject/20120447/ProteinDataset/SwissProt/terms.pkl
terms number:19361
100%|█████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 68.39it/s]
Fold 0 contains 3010 zero shot terms
Fold 1 contains 2925 zero shot terms
Fold 2 contains 2977 zero shot terms
100%|█████████████████████████████████████████████████████████████████████| 47340/47340 [00:00<00:00, 231638.94it/s]
100%|████████████████████████████████████████████████████████████████████████| 47340/47340 [06:31<00:00, 120.94it/s]
Rank of your embeddings is 768
Data Loading Finished!
Start Cross-Validation Fold :0 ...
initialize network with xavier
Training: 100%|████████████████████████████████| 30/30 [34:42<00:00, 69.43s/it, Fold=0, epoch=29, train loss=0.0013]
Evaluate Our Model ...
2025-06-03 00:33:35,159 - BioTrainer.py[line:119] - INFO: fold:0 ont: bp roc auc:0.7682990962892405
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 75.29880478087676% terms is greater than 0.65
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 70.34718269778098% terms is greater than 0.7
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 63.63118952760504% terms is greater than 0.75
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 55.66306203756485% terms is greater than 0.8
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 47.97951052931181% terms is greater than 0.85
2025-06-03 00:33:35,160 - BioTrainer.py[line:121] - INFO: fold:0 ont: bp the AUROC of 24.132043255549164% terms is greater than 0.95
2025-06-03 00:33:37,999 - BioTrainer.py[line:119] - INFO: fold:0 ont: mf roc auc:0.8065591305690799
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 79.19799498746868% terms is greater than 0.65
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 74.18546365914806% terms is greater than 0.7
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 68.79699248120339% terms is greater than 0.75
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 64.0350877192988% terms is greater than 0.8
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 56.01503759398536% terms is greater than 0.85
2025-06-03 00:33:38,000 - BioTrainer.py[line:121] - INFO: fold:0 ont: mf the AUROC of 33.58395989974932% terms is greater than 0.95
2025-06-03 00:33:39,590 - BioTrainer.py[line:119] - INFO: fold:0 ont: cc roc auc:0.8185682792368487
2025-06-03 00:33:39,590 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 82.19780219780195% terms is greater than 0.65
2025-06-03 00:33:39,590 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 79.56043956043932% terms is greater than 0.7
2025-06-03 00:33:39,590 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 74.50549450549428% terms is greater than 0.75
2025-06-03 00:33:39,591 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 69.45054945054925% terms is greater than 0.8
2025-06-03 00:33:39,591 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 61.538461538461355% terms is greater than 0.85
2025-06-03 00:33:39,591 - BioTrainer.py[line:121] - INFO: fold:0 ont: cc the AUROC of 31.20879120879113% terms is greater than 0.95
Start Cross-Validation Fold :1 ...
initialize network with xavier
Training: 100%|███████████████████████████████| 30/30 [33:46<00:00, 67.55s/it, Fold=1, epoch=29, train loss=0.00134]
Evaluate Our Model ...
2025-06-03 01:08:02,652 - BioTrainer.py[line:119] - INFO: fold:1 ont: bp roc auc:0.7669666308890611
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 73.59516616314362% terms is greater than 0.65
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 68.21752265861174% terms is greater than 0.7
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 62.17522658610402% terms is greater than 0.75
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 54.86404833836967% terms is greater than 0.8
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 46.16314199395855% terms is greater than 0.85
2025-06-03 01:08:02,652 - BioTrainer.py[line:121] - INFO: fold:1 ont: bp the AUROC of 24.712990936556125% terms is greater than 0.95
2025-06-03 01:08:05,480 - BioTrainer.py[line:119] - INFO: fold:1 ont: mf roc auc:0.8030036416725372
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 79.61859356376519% terms is greater than 0.65
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 74.73182359952217% terms is greater than 0.7
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 68.65315852204915% terms is greater than 0.75
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 61.26340882002313% terms is greater than 0.8
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 55.30393325387311% terms is greater than 0.85
2025-06-03 01:08:05,480 - BioTrainer.py[line:121] - INFO: fold:1 ont: mf the AUROC of 31.227651966627015% terms is greater than 0.95
2025-06-03 01:08:06,948 - BioTrainer.py[line:119] - INFO: fold:1 ont: cc roc auc:0.8281338585934038
2025-06-03 01:08:06,948 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 84.22273781902588% terms is greater than 0.65
2025-06-03 01:08:06,948 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 79.58236658932746% terms is greater than 0.7
2025-06-03 01:08:06,948 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 74.01392111368936% terms is greater than 0.75
2025-06-03 01:08:06,948 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 67.05336426914174% terms is greater than 0.8
2025-06-03 01:08:06,949 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 59.8607888631092% terms is greater than 0.85
2025-06-03 01:08:06,949 - BioTrainer.py[line:121] - INFO: fold:1 ont: cc the AUROC of 36.89095127610204% terms is greater than 0.95
Start Cross-Validation Fold :2 ...
initialize network with xavier
Training: 100%|███████████████████████████████| 30/30 [34:07<00:00, 68.25s/it, Fold=2, epoch=29, train loss=0.00136]
Evaluate Our Model ...
2025-06-03 01:43:00,542 - BioTrainer.py[line:119] - INFO: fold:2 ont: bp roc auc:0.7852172962569931
2025-06-03 01:43:00,542 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 78.55917667238488% terms is greater than 0.65
2025-06-03 01:43:00,542 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 73.52773013150433% terms is greater than 0.7
2025-06-03 01:43:00,542 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 67.12407089765635% terms is greater than 0.75
2025-06-03 01:43:00,542 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 58.6049170954836% terms is greater than 0.8
2025-06-03 01:43:00,543 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 48.82790165809071% terms is greater than 0.85
2025-06-03 01:43:00,543 - BioTrainer.py[line:121] - INFO: fold:2 ont: bp the AUROC of 25.786163522012732% terms is greater than 0.95
2025-06-03 01:43:03,614 - BioTrainer.py[line:119] - INFO: fold:2 ont: mf roc auc:0.8162966711506128
2025-06-03 01:43:03,614 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 82.41758241758214% terms is greater than 0.65
2025-06-03 01:43:03,614 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 77.53357753357727% terms is greater than 0.7
2025-06-03 01:43:03,614 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 71.42857142857119% terms is greater than 0.75
2025-06-03 01:43:03,615 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 64.7130647130645% terms is greater than 0.8
2025-06-03 01:43:03,615 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 57.75335775335756% terms is greater than 0.85
2025-06-03 01:43:03,615 - BioTrainer.py[line:121] - INFO: fold:2 ont: mf the AUROC of 34.432234432234324% terms is greater than 0.95
2025-06-03 01:43:05,139 - BioTrainer.py[line:119] - INFO: fold:2 ont: cc roc auc:0.8312119549544904
2025-06-03 01:43:05,139 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 84.59657701711429% terms is greater than 0.65
2025-06-03 01:43:05,139 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 80.92909535452264% terms is greater than 0.7
2025-06-03 01:43:05,139 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 77.0171149144249% terms is greater than 0.75
2025-06-03 01:43:05,139 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 69.1931540342294% terms is greater than 0.8
2025-06-03 01:43:05,139 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 60.880195599021675% terms is greater than 0.85
2025-06-03 01:43:05,140 - BioTrainer.py[line:121] - INFO: fold:2 ont: cc the AUROC of 37.163814180929066% terms is greater than 0.95


